<br>
<br>
<br>
<br>
<br>
<h1 style='text-align: center; 
          font-family:courier;
          font-size:3em;'> 
    Limpieza y preprocesamiento de datos con Pandas
</h1>
<br>
<br>
<h4 style='text-align:left; 
          font-family:courier;
          font-size:1.5em;'>
    Inferencia estadística<br>
    Por: Jorge Iván Reyes Hernández
</h4>
<br>
<br>
<br>
<br>

# Introducción
- Pandas es la libreria por default para trabajar con conjuntos de datos grandes (cuyo peso sea menor o igual a 1 GB) usando Python.

- Su nombre viene de "PANel DAta", que hace referencia a datos tabulares (que son los que generalmente se trabajan usando esta libreria).

- Pandas es ampliamente usado, por ejemplo, fue usado durante la construcción de la primera imagen de un agujero negro. [click aquí](https://solarsystem.nasa.gov/resources/2319/first-image-of-a-black-hole/)


# Instalación

Si cuentas con una instalación de Python desde Anaconda es probable que ya tengas instalado Pandas. Por otro lado, si descargaste Python desde su sitio oficial, desde brew o apt, puedes instalar la última versión estable de esta libreria usando el comando
    
    pip install pandas
    
desde la línea de comandos (y una vez activado un ambiente virtual). Si no tienes instalado un intérprete de Python o no tienes algún ambiente virtual da click [aquí](https://www.notion.so/ivanpy/Python-1e57d3105f3d4989835e363b5e19d63a) para más información.

# El ciclo de vida de los datos
<img title="a title" alt="Alt text" src="./data_life_cycle.png">
Fuente: Badia, Antonio. SQL for Data Science, Springer.

# Creación o lectura de datos

Existen diversas formas de cargar datos a Pandas. Por ejemplo, podemos definir en el código los datos en un diccionario o podemos cargar datos preexisten desde un archivo .csv, .xlsl, etc.

## Creación de un dataframe desde un diccionario


In [ ]:
# Importamos la libreria

import pandas as pd


In [ ]:
# Creamos un diccionario con los datos

dict_to_df: dict = {"name": ["Bob", "Mary", "Mita"],
                    "account": [123846, 123972, 347209],
                    "balance": [123, 3972, 7209]}


In [ ]:
# Lo convertimos a un dataframe/tabla

df_1 = pd.DataFrame(dict_to_df)
df_1


En el ejemplo anterior, el constructor **DataFrame** acepta un diccionario de datos y lo convierte en un objeto *frame.DataFrame*, esto es, una tabla. Las llaves (keys) del diccionario (en este caso "name", "account" y "balance") son tomadas como columnas de la tabla, mientras que los valores correspondientes a dichas llaves son tomados como las respectivas observaciones de cada columna.

## Creación de un dataframe desde un archivo

Para crear un DataFrame desde un archivo persistente (local) necesitamos la ruta absoluta (o relativa a nuestro script/notebook) de dicho archivo. El directorio que contiene este notebook viene acompañado de dos archivos (además del propio notebook): "top_movies.csv" y "top_movies.xlsx", scrapeados (usando PowerBI, cosa que veremos más adelante) de [Top 250 Movies](https://www.imdb.com/chart/top).

In [ ]:
from os.path import join

# Ruta relativa de los archivos

filename_1: str = join(".", "top_movies.csv") 
filename_2: str = join(".", "top_movies.xlsx")


Obs: Recuérdese que el puntito "." hace referencia al directorio actual. Entonces "join(".", "top_movies.csv")" crea "./top_movies.csv" o ".\top_movies.csv", dependiendo del OS que tengamos, esto es, la ruta (path) del archivo que queremos cargar.

Para cargar/leer el archivo separado por comas (csv) usando Pandas, usamos la siguiente función.

In [ ]:
df = pd.read_csv(filename_1, sep=",", encoding_errors="ignore")
df


Para cargarlo usando un archivo de Excel primero debemos instalar la libreria *openpyxl*.

In [ ]:
df_2 = pd.read_excel(filename_2)
df_2


Como puede verse, los datos están sucios, en el sentido de que hay información no relevante o incluso errores provenientes de la fuente (cosa usual al momento de hacer [web scraping](https://es.wikipedia.org/wiki/Web_scraping)).

Lo primero que hacemos será limpiar y preprocesar los datos para aumentar su usabilidad. En lo sucesivo usaremos el dataframe "df", pues para los demás el tratamiento es análogo.

# Limpieza y preprocesamiento de datos

Una vez que tenemos nuestros datos en una tabla (dataframe) es usual explorar algunos de ellos (pues podrían ser cientos, miles o millones de datos) usando el método *.head(n)*, donde $n$ hace referencia a cuántos datos queremos mostrar.

In [ ]:
df.head(15)

In [ ]:
# Adicionalmente observamos el tipo de datos que contiene nuestra tabla

df.dtypes


Con esto podemos observar varios problemas. El primero está en la columna llamada "Rank & Title", pues contiene símbolos no deseados, probablemente de errores de codificación al momento de la obtención de los datos. Para este caso particular observamos la ocurrencia de los símbolos "\r\n", que procederemos a quitar.

A continuación se define una función que toma una cadena y la devuelve sin los símbolos no deseados.

In [ ]:
def remove_symbols(string):
    return string.replace("\r\n", "")


In [ ]:
print(remove_symbols("\r\n test \r\n string"))


Ahora se la aplicamos a toda la columna "Rank & Title" usando el método *.apply(fn)*, donde *fn* es la función que le vamos a aplicar a cada elemento de la colunma deseada.

In [ ]:
rank_title = df["Rank & Title"].apply(remove_symbols)
rank_title


Podemos notar que aunque ya no están esos símbolos, ahora tenemos demasidos espacios. Los quitaremos usando expresiones regulares.

In [ ]:
import re


def remove_spaces(string):
    # Quitamos espacios internos
    string = re.sub(' +', ' ', string)
    
    # Quitamos espacios a los extremos
    string = string.lstrip().strip()
    
    return string


In [ ]:
print(remove_spaces(" test     string  "))


In [ ]:
rank_title = rank_title.apply(remove_spaces)
rank_title


In [ ]:
# Reemplazamos la columna inicial por la que está limpia
df["Rank & Title"] = rank_title


In [ ]:
df.head(15)


In [ ]:
# Eliminamos las dos columnas que no deseamos
df.drop(['Your Rating', '_1'], axis=1, inplace=True)


In [ ]:
df.head(15)


In [ ]:
# Separamos las columnas
df[['Rank', 'Title_Year']] = df['Rank & Title'].str.split('.', 1, expand=True)

# Retiramos la columna no deseada
df.drop('Rank & Title', axis=1, inplace=True)


In [ ]:
df.head(15)


In [ ]:
# Separamos las columnas
df[['Title', 'Year']] = df['Title_Year'].str.split('(', 1, expand=True)

# Retiramos la columna no deseada
df.drop('Title_Year', axis=1, inplace=True)


In [ ]:
df.head(15)

In [ ]:
# Removemos el parentesis que sobra y convertimos a número
def remove_parenthesis(string):
    return int(string.replace(")", ""))


In [ ]:
df['Year']= df["Year"].apply(remove_parenthesis)


In [ ]:
df.head(15)

In [ ]:
# Reordenamos para una mejor visualización
df = df[["Rank", "Title", "Year", "IMDb Rating"]]

df.head(15)

In [ ]:
df.dtypes

In [ ]:
# Convertimos en el tipo adecuado de dato
df['Rank'] = pd.to_numeric(df['Rank'])


In [ ]:
df.dtypes

Para saber más sobre los tipos de datos en Pandas da click [aquí](https://pbpython.com/pandas_dtypes.html).

In [ ]:
df.head(15)

# Hacemos persistentes nuestros datos
Para persistir nuestros datos ya limpios y preprocesados, podemos almacenarlo en algún formato deseado, por ejemplo .csv.
En Pandas esto se hace con el método 
    
    .to_csv()

In [ ]:
filename_to_save = "top_movies_cleaned.csv"

df.to_csv(filename_to_save, index=False)

**Referencias**
- https://pandas.pydata.org/pandas-docs/stable/reference/io.html
- Badia, Antonio. SQL for Data Science, Springer.
- Stepanek, Hannah. Thinking in Pandas, Apress.